In [2]:
from urllib.request import Request, urlopen
from requests import get
from bs4 import BeautifulSoup
from pandas import DataFrame, read_csv, isnull, read_html
import json
import re

In [115]:
stateNums = {"Iowa": ["19", "197"], "Illinois": ["17", "203"], "Indiana": ["18", "183"], "Kansas": ["20", "209"], "Michigan": ["26", "165"], 
            "Missouri": ["29","229"], "Minnesota": ["27", "173"], "Nebraska": ["31", "185"], "North Dakota": ["38", "105"], "Ohio": ["39", "175"],
            "South Dakota": ["46", "137"], "Wisconsin": ["55", "141"]}
itemData = {"State": [], "County": [], 'Crop Insurance Subsidies': [], 'Conservation Subsidies': [], 'Commodity Subsidies': [], 'Total Subsidies': [], 'Disaster Subsidies': []}
cats = []

for state in stateNums:
    url = f"https://farm.ewg.org/region.php?fips={stateNums[state][0]}000"
    req = Request(url,headers={'User-Agent': 'Mozilla/5.0'})
    response = urlopen(req).read()
    soup = BeautifulSoup(response, 'html.parser')
    # print(soup)
    data = [j.text for j in soup.find_all("script") if len(j.text) > 1000][1]
    # print(data)
    filtered = re.findall("{.+},", data.split("areas: ")[1])[0].split("},")
    print(len(filtered))
    for txt in filtered:
#         print(txt)
        a=(txt.split(", customData: "))[0]
        county = a.split("title:")[-1].split(",")[0].replace('"',"")
        if not(txt) or not(county):continue
    #     print(txt)
#         print(a)
        
    #     print(county)
        tdf = pd.read_html(txt)[0]
    #     print(tdf)
        itemData["State"].append(state)
        itemData["County"].append(county)
        for cat, amount in zip(tdf["Subsidy Category"], tdf["Subsidies 1995-2020"]):                        
            itemData[cat].append(float(amount.replace("]", "").replace("$", "").replace(",", "")))
        
df = DataFrame(itemData)
print(df)
print(set(cats))


99
102
93
111
83
115
88
129
53
88
67
72
          State             County  Crop Insurance Subsidies  \
0          Iowa       Adair County                53025842.0   
1          Iowa       Adams County                39718999.0   
2          Iowa   Allamakee County                26199101.0   
3          Iowa   Appanoose County                35960985.0   
4          Iowa     Audubon County                55663791.0   
...         ...                ...                       ...   
1041  Wisconsin  Washington County                15837117.0   
1042  Wisconsin    Waukesha County                16379450.0   
1043  Wisconsin     Waupaca County                30480459.0   
1044  Wisconsin    Waushara County                38254976.0   
1045  Wisconsin   Winnebago County                28273045.0   

      Conservation Subsidies  Commodity Subsidies  Total Subsidies  \
0                 76147154.0          165440361.0      300673583.0   
1                 75052996.0          107220689.0  

In [117]:
df.to_csv("ewgData.csv")

In [38]:
stateNums = {"Iowa": ["19", "197"], "Illinois": ["17", "203"], "Indiana": ["18", "183"], "Kansas": ["20", "209"], "Michigan": ["26", "165"], 
            "Missouri": ["29","229"], "Minnesota": ["27", "173"], "Nebraska": ["31", "185"], "North Dakota": ["38", "105"], "Ohio": ["39", "175"],
            "South Dakota": ["46", "137"], "Wisconsin": ["55", "141"]}
subsidyData = {"County": [], "Subsidy Type": [], "Subsidy Sub_Type": [], "Amount": []}
for state in stateNums:
    for num in range(1, int(stateNums[state][1]), 2):
        url = f"https://farm.ewg.org/region.php?fips={stateNums[state][0]}{'0'*(3-len(str(num)))}{num}"
        print(url)
        req = Request(url,headers={'User-Agent': 'Mozilla/5.0'})
        response = urlopen(req).read()
        soup = BeautifulSoup(response, 'html.parser')
        county = soup.find("div", id="main_content_area").find("h2").text.split(",")[1]
        initialDF = read_html(response)[0]
        links = [a["href"] for a in soup.find("table").find_all("a")]
        for i, link in enumerate(links):
            print(i)
            program = initialDF["Program"][i]
            newURL = f"https://farm.ewg.org/{link}"
            newReq = Request(newURL,headers={'User-Agent': 'Mozilla/5.0'})
            newResponse = urlopen(newReq).read()
            try:
                tdf = read_html(newResponse)[0]
            except ValueError:
                continue
            subsidyData["County"]+=[county]*len(tdf["Program"])
            subsidyData["Subsidy Type"]+=[program]*len(tdf["Program"])
            subsidyData["Subsidy Sub_Type"]+=list(tdf["Program"])
            subsidyData["Amount"]+=[int(n.replace(",","").replace("$","")) for n in tdf["Total Payments1995-2020‡"]]
    #     print(tdf)
    
print(DataFrame(subsidyData))

https://farm.ewg.org/region.php?fips=19001
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19003
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19005
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19007
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19009
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19011
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19013
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19015
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19017
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19019
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19021
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19023
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19025
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19027
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19029
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19031
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19033
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19035
0
1
2
3
4
https://farm.ewg.org/region.php?fips=19037
0
1

0
1
2
3
4
https://farm.ewg.org/region.php?fips=17115
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17117
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17119
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17121
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17123
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17125
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17127
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17129
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17131
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17133
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17135
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17137
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17139
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17141
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17143
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17145
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17147
0
1
2
3
4
https://farm.ewg.org/region.php?fips=17149
0
1
2
3
4
https://farm.ewg.org/region.php?fips

0
1
2
3
4
https://farm.ewg.org/region.php?fips=20041
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20043
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20045
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20047
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20049
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20051
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20053
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20055
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20057
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20059
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20061
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20063
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20065
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20067
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20069
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20071
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20073
0
1
2
3
4
https://farm.ewg.org/region.php?fips=20075
0
1
2
3
4
https://farm.ewg.org/region.php?fips

0
1
2
3
4
https://farm.ewg.org/region.php?fips=26143
0
1
2
3
4
https://farm.ewg.org/region.php?fips=26145
0
1
2
3
4
https://farm.ewg.org/region.php?fips=26147
0
1
2
3
4
https://farm.ewg.org/region.php?fips=26149
0
1
2
3
4
https://farm.ewg.org/region.php?fips=26151
0
1
2
3
4
https://farm.ewg.org/region.php?fips=26153
0
1
2
3
4
https://farm.ewg.org/region.php?fips=26155
0
1
2
3
4
https://farm.ewg.org/region.php?fips=26157
0
1
2
3
4
https://farm.ewg.org/region.php?fips=26159
0
1
2
3
4
https://farm.ewg.org/region.php?fips=26161
0
1
2
3
4
https://farm.ewg.org/region.php?fips=26163
0
1
2
3
4
https://farm.ewg.org/region.php?fips=29001
0
1
2
3
4
https://farm.ewg.org/region.php?fips=29003
0
1
2
3
4
https://farm.ewg.org/region.php?fips=29005
0
1
2
3
4
https://farm.ewg.org/region.php?fips=29007
0
1
2
3
4
https://farm.ewg.org/region.php?fips=29009
0
1
2
3
4
https://farm.ewg.org/region.php?fips=29011
0
1
2
3
4
https://farm.ewg.org/region.php?fips=29013
0
1
2
3
4
https://farm.ewg.org/region.php?fips

2
3
4
https://farm.ewg.org/region.php?fips=27057
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27059
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27061
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27063
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27065
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27067
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27069
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27071
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27073
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27075
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27077
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27079
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27081
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27083
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27085
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27087
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27089
0
1
2
3
4
https://farm.ewg.org/region.php?fips=27091
0
1
2
3
4
https://farm.ewg.org/region.php?fips=270

0
1
2
3
4
https://farm.ewg.org/region.php?fips=38011
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38013
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38015
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38017
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38019
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38021
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38023
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38025
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38027
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38029
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38031
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38033
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38035
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38037
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38039
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38041
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38043
0
1
2
3
4
https://farm.ewg.org/region.php?fips=38045
0
1
2
3
4
https://farm.ewg.org/region.php?fips

0
1
2
3
4
https://farm.ewg.org/region.php?fips=46041
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46043
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46045
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46047
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46049
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46051
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46053
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46055
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46057
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46059
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46061
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46063
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46065
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46067
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46069
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46071
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46073
0
1
2
3
4
https://farm.ewg.org/region.php?fips=46075
0
1
2
3
4
https://farm.ewg.org/region.php?fips

In [39]:
df = DataFrame(subsidyData)
df.to_csv("ewgData.csv")